In [12]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
from torchvision import datasets, transforms
!pip install torchsummary

from torchsummary import summary

In [17]:
class ConstructNet(nn.Module):
    def __init__(self, DNA, loss_fn, input_size, output_size):
        #print(DNA)
        super(ConstructNet, self).__init__()
        self.DNA = DNA
        self.input_size = input_size
        self.output_size = output_size
        self.layers = []
        self.loss_fn = loss_fn

        # Append first layer
        self.layers.append(nn.Conv2d(1, 2 * self.DNA[0][1], kernel_size=3, stride=1, padding=1))
        self.layers.append(nn.ReLU())
        
        for i in range(1, len(self.DNA)):
            #print(self.DNA[i])
            if self.DNA[i][0] == "C":
                self.layers.append(nn.Conv2d(2 * self.DNA[i-1][1], 2 * self.DNA[i][1], kernel_size=3, stride=1, padding=1))
                self.layers.append(nn.ReLU())
            if self.DNA[i][0] == "D":
                # The input size is the output of the last layer
                tmp_input_size = self.last_layer_output_size()
                self.layers.append(nn.Linear(tmp_input_size, self.DNA[i][1]))
                self.layers.append(nn.ReLU())
            if self.DNA[i][0] == "R":
                self.layers.append(nn.Dropout(self.DNA[i][1]))

        # Append the output layer 
        self.layers.append(nn.Flatten())
        
        cnn_output = self.cnn_output_size()
        self.layers.append(nn.Linear(cnn_output[1], self.output_size*10))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(self.output_size*10, self.output_size))
        
        self.net = nn.Sequential(*self.layers, nn.Softmax(dim=1))
    def cnn_output_size(self):
        input = torch.ones(64, 1, self.input_size,self.input_size)
        #print(input.shape)
        for layer in self.layers:
            input = layer(input)
        #print(self.layers)
        return input.shape
    
    def forward(self, x):
        #print(x.shape)
        return self.net(x)

    '''
    Based on the layers created, find the output size of the last dense layer
    '''

    def last_layer_output_size(self):
        for layer in self.layers[::-1]:
            if isinstance(layer, nn.Linear):
                return layer.out_features

    def train_net(self, device, train_loader, optimizer, epoch, log_interval, print_stats):
        self.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = self.forward(data)
            #print(output)
            loss = self.loss_fn(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0 and print_stats:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))

    def test(self, device, test_loader, print_stats=False):
        self.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = self.forward(data)
                #print(f"output: {output}, target: {target}")
                test_loss += self.loss_fn.forward(output, target).item()  # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        accuracy = correct / len(test_loader.dataset)
        if print_stats:
            print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                test_loss, correct, len(test_loader.dataset),
                100. * accuracy))
        return test_loss, accuracy

    def count_parameters(self):
        # https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/7
        return sum(p.numel() for p in self.net.parameters() if p.requires_grad)        

In [21]:
best_net = torch.load('./models/gen_54/ind_25_314806_0.9431.pt', map_location=torch.device('cpu'))

In [22]:
summary(best_net,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 2, 28, 28]              20
              ReLU-2            [-1, 2, 28, 28]               0
            Conv2d-3            [-1, 4, 28, 28]              76
              ReLU-4            [-1, 4, 28, 28]               0
           Flatten-5                 [-1, 3136]               0
            Linear-6                  [-1, 100]         313,700
              ReLU-7                  [-1, 100]               0
            Linear-8                   [-1, 10]           1,010
           Softmax-9                   [-1, 10]               0
Total params: 314,806
Trainable params: 314,806
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.10
Params size (MB): 1.20
Estimated Total Size (MB): 1.30
-------------------------------------------